In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import util.common as util

# Load Data

In [3]:
clean_dir = "/project/data/cicids2017/clean/"
x_binary_train, y_binary_train, x_binary_val, y_binary_val, x_binary_test, y_binary_test, x_multi_train, y_multi_test = util.load_data(clean_dir, sample_size=1948)

(D)DOS          321637
Port Scan        90694
Brute Force       9150
Web Attack        2143
Botnet            1948
Infiltration        36
Heartbleed          11
Name: Label, dtype: int64
Attack type:    #Original:     #Sampled:      #Train:       #Test:
      (D)DOS        321637          1948         1363          585
      Botnet          1948          1948         1363          585
 Brute Force          9150          1948         1363          585
  Heartbleed            11            11            0           11
Infiltration            36            36            0           36
   Port Scan         90694          1948         1363          585
  Web Attack          2143          1948         1363          585


In [4]:
np.unique(y_binary_train, return_counts=True)

(array([1.]), array([100000]))

In [5]:
np.unique(y_binary_val, return_counts=True)

(array([-1.,  1.]), array([  6815, 100000]))

In [6]:
np.unique(y_binary_test, return_counts=True)

(array([-1.,  1.]), array([ 2972, 30000]))

## Normalise data

In [145]:
from sklearn.preprocessing import QuantileTransformer

binary_scaler = QuantileTransformer(output_distribution='normal')
x_binary_train_s = binary_scaler.fit_transform(x_binary_train)
x_binary_val_s = binary_scaler.transform(x_binary_val)
x_binary_test_s = binary_scaler.transform(x_binary_test)

In [8]:
from sklearn.preprocessing import MinMaxScaler

binary_scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
x_binary_train_s = binary_scaler.fit_transform(x_binary_train)
x_binary_val_s = binary_scaler.transform(x_binary_val)
x_binary_test_s = binary_scaler.transform(x_binary_test)

# Train Model

In [126]:
import tensorflow as tf
from keras.regularizers import l2
from keras.models import Model, load_model
from keras.layers import Input, Dense, Layer, InputSpec
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers, activations, initializers, constraints, Sequential
from keras import backend as K
from keras.constraints import UnitNorm, Constraint

class DenseTied(Layer):
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 tied_to=None,
                 **kwargs):
        self.tied_to = tied_to
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super().__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True
                
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        if self.tied_to is not None:
            self.kernel = self.tied_to.kernel
            self._non_trainable_weights.append(self.kernel)
        else:
            self.kernel = self.add_weight(shape=(input_dim, self.units),
                                          initializer=self.kernel_initializer,
                                          name='kernel',
                                          regularizer=self.kernel_regularizer,
                                          constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint,
                                        trainable=True)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def call(self, inputs):
        # Transpose only here because of tf2 eager mode 
        # https://stackoverflow.com/questions/53751024/tying-autoencoder-weights-in-a-dense-keras-layer
        output = K.dot(inputs, K.transpose(self.kernel))
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

In [169]:
def create_model(params):
    input_layer = Input(shape=(params["input_dimension"],))
    model = input_layer
    
    # Encoder
    encoder = []
    for n in params["n_neurons"]:
        encoder_layer = Dense(n, activation=params['hidden_activation'], activity_regularizer=l2(params["l2_reg"]))
        encoder.append(encoder_layer)
        model = encoder_layer(model)

    # Decoder - Do not repeat encoded layer
    for n, encoder_layer in zip(reversed(params["n_neurons"][:-1]), reversed(encoder[1:])):
        decoder_layer = DenseTied(n, activation=params['hidden_activation'], tied_to=encoder_layer, activity_regularizer=l2(params["l2_reg"]))
        model = decoder_layer(model)

    # Output Layer
    model = DenseTied(params["input_dimension"], activation=params['output_activation'], tied_to=encoder[0], activity_regularizer=l2(params["l2_reg"]))(model)
#     model = Dense(params["input_dimension"], activation=params['output_activation'], activity_regularizer=l2(params["l2_reg"]))(model)
    autoencoder = Model(inputs=input_layer, outputs=model)
    autoencoder.compile(optimizer=params['optimizer'], loss=params['loss'])
    return autoencoder

In [171]:
params = {'scaler': 'quantile', 'output_activation': 'linear', 'hidden_activation': 'relu', 'optimizer': 'adam', 'loss': 'mean_squared_error', 'input_dimension': 67, 'n_layers': 3, 'n_neurons': [66, 63, 10], 'l2_reg': 1.271016e-03}

In [172]:
m = create_model(params)
m.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 67)]              0         
_________________________________________________________________
dense_63 (Dense)             (None, 66)                4488      
_________________________________________________________________
dense_64 (Dense)             (None, 63)                4221      
_________________________________________________________________
dense_65 (Dense)             (None, 10)                640       
_________________________________________________________________
dense_tied_49 (DenseTied)    (None, 63)                703       
_________________________________________________________________
dense_tied_50 (DenseTied)    (None, 66)                4287      
_________________________________________________________________
dense_tied_51 (DenseTied)    (None, 67)                4555

In [146]:
history = m.fit(
    x_binary_train_s,
    x_binary_train_s,
    epochs=4, 
    shuffle=True,
    verbose=1
)

Epoch 1/4
3125/3125 [==============================] - 3s 1ms/step - loss: 8.4714
Epoch 2/4
3125/3125 [==============================] - 3s 1ms/step - loss: 2.0686
Epoch 3/4
3125/3125 [==============================] - 3s 1ms/step - loss: 1.4958
Epoch 4/4
3125/3125 [==============================] - 3s 1ms/step - loss: 1.3056


In [147]:
x_val_autoencoder = m.predict(x_binary_val_s)
val_score = util.anomaly_scores(x_binary_val_s, x_val_autoencoder)
val_metrics = util.evaluate_results(y_binary_val, val_score)
val_metrics

/project/Two-Stage/util/common.py:97: RuntimeWarning: invalid value encountered in true_divide
  results["f1"] = 2*precision*recall/(precision+recall)
/project/Two-Stage/util/common.py:98: RuntimeWarning: invalid value encountered in true_divide
  results["f2"] = 5*precision*recall/(4*precision+recall)


f1                   precision     0.291843
                     recall        0.496112
                     f1            0.367500
                     f2            0.435191
f2                   precision     0.187044
                     recall        0.832135
                     f1            0.305434
                     f2            0.492454
threshold                         39.555313
au_precision_recall                0.216846
auroc                              0.835452
dtype: float64

In [142]:
model = load_model("results/binary/autoencoder/TWOS-37/models/model_1223.h5")

In [143]:
model.summary()

Model: "model_305"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_306 (InputLayer)       [(None, 67)]              0         
_________________________________________________________________
dense_1322 (Dense)           (None, 66)                4488      
_________________________________________________________________
dense_1323 (Dense)           (None, 63)                4221      
_________________________________________________________________
dense_1324 (Dense)           (None, 66)                4224      
_________________________________________________________________
dense_1325 (Dense)           (None, 67)                4489      
Total params: 17,422
Trainable params: 17,422
Non-trainable params: 0
_________________________________________________________________


In [148]:
x_val_autoencoder = model.predict(x_binary_val_s)
val_score = util.anomaly_scores(x_binary_val_s, x_val_autoencoder)
val_metrics = util.evaluate_results(y_binary_val, val_score)
val_metrics

/project/Two-Stage/util/common.py:97: RuntimeWarning: invalid value encountered in true_divide
  results["f1"] = 2*precision*recall/(precision+recall)
/project/Two-Stage/util/common.py:98: RuntimeWarning: invalid value encountered in true_divide
  results["f2"] = 5*precision*recall/(4*precision+recall)


f1                   precision     0.301306
                     recall        0.758327
                     f1            0.431260
                     f2            0.581825
f2                   precision     0.289229
                     recall        0.802201
                     f1            0.425166
                     f2            0.592154
threshold                         17.202332
au_precision_recall                0.318725
auroc                              0.907273
dtype: float64

In [10]:
from util.AUROCEarlyStoppingPruneCallback import AUROCEarlyStoppingPruneCallback

autoencoder = create_model(x_binary_train_s.shape[1], 3, [51, 48, 32], 1e-4)
history = autoencoder.fit(
    x_binary_train_s,
    x_binary_train_s,
    epochs=5, 
    shuffle=True,
    verbose=1
)
# trial.set_user_attr('epochs', len(history.history['loss']))
# trial.set_user_attr('losses', history.history['loss'])
x_val_autoencoder = autoencoder.predict(x_binary_val_s)
val_score = util.anomaly_scores(x_binary_val_s, x_val_autoencoder)
val_metrics = util.evaluate_results(y_binary_val, val_score)
val_metrics

Epoch 1/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0214
Epoch 2/5
3125/3125 [==============================] - 5s 2ms/step - loss: 0.0014
Epoch 3/5
3125/3125 [==============================] - 5s 2ms/step - loss: 9.7788e-04
Epoch 4/5
3125/3125 [==============================] - 5s 2ms/step - loss: 8.0913e-04
Epoch 5/5
3125/3125 [==============================] - 5s 2ms/step - loss: 7.1869e-04


/project/Two-Stage/util/common.py:97: RuntimeWarning: invalid value encountered in true_divide
  results["f1"] = 2*precision*recall/(precision+recall)


precision              0.128594
recall                 0.892443
f1                     0.224797
threshold              0.002786
au_precision_recall    0.215373
auroc                  0.750486
Name: 20800, dtype: float64

In [21]:
val_metrics

precision                0.090092
recall                   0.604255
f1                       0.156805
threshold              704.071541
au_precision_recall      0.064433
auroc                    0.522964
Name: 59652, dtype: float64

In [24]:
val_metrics

precision               0.392387
recall                  0.319149
f1                      0.351999
threshold              14.375418
au_precision_recall     0.242733
auroc                   0.803078
Name: 86980, dtype: float64

In [15]:
import neptune
neptune.init(project_qualified_name='verkerken/two-stage-binary', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMGJlYTgzNzEtM2U3YS00ODI5LWEzMzgtM2M0MjcyMDIxOWUwIn0=')

Project(verkerken/two-stage-binary)

In [16]:
neptune.create_experiment('hello-world-ae', tags=["autoencoder", "cicids2017", "binary"])

https://ui.neptune.ai/verkerken/two-stage-binary/e/TWOS-2


Experiment(TWOS-2)

In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(-y_binary_val, val_score)

0.735831134262656

In [24]:
str(val_metrics.to_dict())

"{'precision': 0.12859438430311232, 'recall': 0.8924431401320616, 'f1': 0.22479717617489975, 'threshold': 0.0027857437069087783, 'au_precision_recall': 0.21537287814713807, 'auroc': 0.7504856258253851}"

In [57]:
import os


In [62]:
import pathlib
pathlib.Path('results/binary/autoencoder/TWOS-3').mkdir(parents=True, exist_ok=True)

In [71]:
import optuna
study_storage = '/project/Two-Stage/results/binary/autoencoder.db'
study = optuna.create_study(
    study_name="test", 
    direction='maximize', 
    storage=f'sqlite:///{study_storage}',
    load_if_exists=True
)

[I 2021-03-02 00:08:38,990] Using an existing study with name 'test' instead of creating a new one.


In [76]:
val_metrics = util.evaluate_results(y_binary_val, val_score)
val_metrics

/project/Two-Stage/util/common.py:97: RuntimeWarning: invalid value encountered in true_divide
  results["f1"] = 2*precision*recall/(precision+recall)
/project/Two-Stage/util/common.py:98: RuntimeWarning: invalid value encountered in true_divide
  max_index = results["f1"].idxmax()


f1                   precision    0.128594
                     recall       0.892443
                     f1           0.224797
                     f2           0.407881
f2                   precision    0.128546
                     recall       0.893617
                     f1           0.224761
                     f2           0.407980
threshold                         0.002786
au_precision_recall               0.215373
auroc                             0.750486
dtype: float64

In [81]:
val_metrics.index = val_metrics.index.map(''.join)
val_metrics

f1precision            0.128594
f1recall               0.892443
f1f1                   0.224797
f1f2                   0.407881
f2precision            0.128546
f2recall               0.893617
f2f1                   0.224761
f2f2                   0.407980
threshold              0.002786
au_precision_recall    0.215373
auroc                  0.750486
dtype: float64

In [82]:
val_metrics.to_dict()

{'f1precision': 0.12859438430311232,
 'f1recall': 0.8924431401320616,
 'f1f1': 0.22479717617489975,
 'f1f2': 0.4078813241053705,
 'f2precision': 0.12854609929078015,
 'f2recall': 0.8936170212765957,
 'f2f1': 0.22476056909818976,
 'f2f2': 0.40798006324025937,
 'threshold': 0.0027857437069087783,
 'au_precision_recall': 0.21537287814713807,
 'auroc': 0.7504856258253851}

In [87]:
for k,v in val_metrics.to_dict().items():
    print(k)
    print(v)

f1precision
0.12859438430311232
f1recall
0.8924431401320616
f1f1
0.22479717617489975
f1f2
0.4078813241053705
f2precision
0.12854609929078015
f2recall
0.8936170212765957
f2f1
0.22476056909818976
f2f2
0.40798006324025937
threshold
0.0027857437069087783
au_precision_recall
0.21537287814713807
auroc
0.7504856258253851


In [29]:
def create_model(params):
    input_layer = Input(shape=(params["input_dimension"],))
    model = input_layer
    
    # Encoder
    for n in params["n_neurons"]:
        model = Dense(n, activation=params['hidden_activation'], activity_regularizer=l2(params["l2_reg"]))(model)

    # Decoder - Do not repeat encoded layer
    for n in reversed(params["n_neurons"][:-1]):
        model = Dense(n, activation=params['hidden_activation'], activity_regularizer=l2(params["l2_reg"]))(model)

    # Output Layer
    model = Dense(params["input_dimension"], activation=params['output_activation'], activity_regularizer=l2(params["l2_reg"]))(model)
    autoencoder = Model(inputs=input_layer, outputs=model)
    autoencoder.compile(optimizer=params['optimizer'], loss=params['loss'])
    return autoencoder

In [26]:
params = {'scaler': 'quantile', 'output_activation': 'linear', 'hidden_activation': 'relu', 'optimizer': 'adam', 'loss': 'mean_squared_error', 'input_dimension': 67, 'n_layers': 1, 'n_neurons': [62, 27, 5], 'l2_reg': 1.6664447908824294e-10}

In [27]:
m = create_model(params)

In [8]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.regularizers import l2

In [28]:
m.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 67)]              0         
_________________________________________________________________
dense_21 (Dense)             (None, 62)                4216      
_________________________________________________________________
dense_22 (Dense)             (None, 27)                1701      
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 140       
_________________________________________________________________
dense_24 (Dense)             (None, 27)                162       
_________________________________________________________________
dense_25 (Dense)             (None, 62)                1736      
_________________________________________________________________
dense_26 (Dense)             (None, 67)                4221

# Log Final results to disk and neptune

In [3]:
import optuna

In [22]:
study_storage = 'results/binary/autoencoder_linear_cpu.db'
# study_storage = 'results/binary/autoencoder_linear_gpu.db'
# study_storage = 'results/binary/autoencoder_sigmoid.db'

In [23]:
list(map(lambda s: s.study_name, optuna.study.get_all_study_summaries(storage=f"sqlite:///{study_storage}")))

['TWOS-36']

In [24]:
study_name = "TWOS-36"
save_dir = f'results/binary/autoencoder/{study_name}'

In [25]:
study = optuna.load_study(study_name=study_name, storage=f"sqlite:///{study_storage}")
results = study.trials_dataframe()
results.sort_values(by=['value'], inplace=True, ascending=False)
results.to_csv(f'{save_dir}/result.csv')
results.head()

,number,value,datetime_start,datetime_complete,duration,params_encoder_layers,params_l2,params_n_layer_0,params_n_layer_1,params_n_layer_2,...,user_attrs_f1f2,user_attrs_f1precision,user_attrs_f1recall,user_attrs_f2f1,user_attrs_f2f2,user_attrs_f2precision,user_attrs_f2recall,user_attrs_losses,user_attrs_threshold,state
231,231,0.905805,2021-03-02 05:13:05.952757,2021-03-02 05:14:18.476888,0 days 00:01:12.524131,1,0.000005,56.0,NaN,NaN,...,0.612734,0.387971,0.716508,0.425675,0.620604,0.279407,0.893324,"[0.32243651151657104, 0.018776101991534233, 0....",0.467211,COMPLETE
511,511,0.895115,2021-03-02 06:52:15.228921,2021-03-02 06:53:39.536480,0 days 00:01:24.307559,1,0.000118,50.0,NaN,NaN,...,0.489228,0.384095,0.525165,0.404010,0.579629,0.268449,0.816141,"[0.4473184645175934, 0.1314028948545456, 0.113...",1.423257,COMPLETE
543,543,0.893542,2021-03-02 07:03:27.219742,2021-03-02 07:05:00.952675,0 days 00:01:33.732933,2,0.000014,58.0,56.0,NaN,...,0.537078,0.394432,0.590462,0.415775,0.576215,0.283988,0.775789,"[0.28489014506340027, 0.05160053074359894, 0.0...",2.397769,COMPLETE
312,312,0.892667,2021-03-02 05:43:29.257399,2021-03-02 05:45:01.059162,0 days 00:01:31.801763,1,0.000040,42.0,NaN,NaN,...,0.581205,0.359484,0.687161,0.471541,0.582853,0.357690,0.691709,"[0.41826558113098145, 0.06229858472943306, 0.0...",0.549583,COMPLETE
71,71,0.890172,2021-03-02 04:17:49.909151,2021-03-02 04:19:01.210221,0 days 00:01:11.301070,1,0.000028,53.0,NaN,NaN,...,0.586113,0.361806,0.693617,0.470428,0.628728,0.331374,0.810565,"[0.33263203501701355, 0.046308618038892746, 0....",0.683945,COMPLETE


In [26]:
import neptune
from neptunecontrib.monitoring.optuna import log_study_info

project = neptune.init(project_qualified_name='verkerken/two-stage-binary', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMGJlYTgzNzEtM2U3YS00ODI5LWEzMzgtM2M0MjcyMDIxOWUwIn0=')
my_exp = project.get_experiments(id=study_name)[0]
log_study_info(study, experiment=my_exp)

In [27]:
from neptunecontrib.api.table import log_table
log_table("results_overview", results, experiment=my_exp)